# Validación Externa de Métodos de Clustering
En este cuaderno, realizaremos una validación externa de los métodos de clustering que hemos aplicado previamente. Utilizaremos un conjunto de etiquetas predefinidas basadas en categorías de diagnóstico para evaluar la calidad de los clusters generados.

In [2]:
import pandas as pd

In [3]:
# Cargar el conjunto de datos
hado_data = pd.read_csv('hado_data_clustering.csv', sep=',')
hado_data.columns

Index(['h_procedencia', 'hospital_category', 's_procedencia',
       'procedencia_category', 'diagnostico', 'diagnosis_category',
       'motivo_ing', 'ingreso_category', 'motivo_alta', 'alta_category',
       'fecha_alta', 'ap', 'n_estancias', 'n_visitas', 'paliativo_onc_noc',
       'paliativo_no_onc_noc', 'fiebre', 'disnea', 'dolor', 'delirium',
       'sedacion', 'p_terminal', 'agonia', 'ast_anorx', 'cronico_reag',
       'trato_antibiotico', 'transfusion', 'paracentesis', 'agudo_estable',
       'toracocentesis', 'fe_iv', 'ps_ecog', 'barthel', 'gds_fast', 'eva_ing',
       'otros_complicaciones', 'otros', 'otros_1', 'otros_2', 'otros_category',
       'categorized_combined_otros', 'tiene_sedacion', 'morfina', 'midazolam',
       'buscapina', 'haloperidol', 'levomepromazina', 'medico', 'ayuntamiento',
       'year', 'diagnostico_cluster', 'motivo_ing_cluster',
       'motivo_alta_cluster', 'diagnostico_cluster_optimal',
       'motivo_ing_cluster_optimal', 'motivo_alta_cluster_opti

## Crear Etiquetas de Diagnóstico Externas
Utilizaremos un diccionario de categorías de diagnóstico para asignar etiquetas externas a cada registro en el conjunto de datos.

* Diccionario de categorías de diagnóstico
```python
diagnosis_categories = {
  'Cardiacas': ['cardiaca', 'corazon', 'miocardio', 'arritmia', 'insuficiencia cardiaca', 'endocarditis'],
  'Pulmonares y respiratorias': ['pulmon', 'neumonia', 'broncoaspiracion', 'bronquiectasias', 'fibrosis pulmonar', 'insuficiencia respiratoria'],
  'Neurologicas': ['demencia', 'ela', 'acv', 'encefalopatia', 'glioblastoma'],
  'Hepaticas y pancreaticas': ['cirrosis', 'hepatocarcinoma', 'colangitis', 'colangiocarcinoma', 'pancreas'],
  'Renales y urinarias': ['ITU', 'infeccion urinaria', 'pielonefritis', 'prostatitis', 'renal'],
  'Canceres y neoplasias': ['cancer', 'tumor', 'glioblastoma', 'linfoma', 'mieloma', 'carcinoma', 'neoplasia', 'hepatocarcinoma', 'colangiocarcinoma'],
  'Hematologicas': ['anemia', 'linfoma', 'mieloma', 'leucemia', 'sindrome mielodisplasico'],
  'Infecciones': ['infeccion', 'sepsis', 'bacteriemia', 'bronquiectasias infectadas', 'endocarditis', 'pielonefritis', 'prostatitis', 'diverticulitis', 'celulitis'],
  'Musculoesqueléticas y de piel': ['amputacion', 'ulcera', 'fractura']
}
```
> Las etiquetas están creadas en la columna `diagnosis_category`

In [4]:
hado_data = hado_data.rename(columns={'diagnostico_cluster_optimal': 'diagnostico_kmeans',
                                      'motivo_ing_cluster_optimal': 'motivo_ing_kmeans',
                                      'motivo_alta_cluster_optimal': 'motivo_alta_kmeans',
                                      'diagnostico_topic': 'diagnostico_lda'
                                     }
                            )

In [5]:
# Importar bibliotecas necesarias para la validación externa
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, homogeneity_score, completeness_score

# Lista para almacenar los resultados de validación externa
external_validation_metrics = []

# Calcular métricas de validación externa para cada método de clustering
for method, label_column in zip(['K-means', 'LDA', 'Hierarchical', 'DBSCAN'],
                                ['diagnostico_kmeans', 'diagnostico_lda', 'diagnostico_hierarchical', 'diagnostico_dbscan']):
    # Excluir clusters con etiqueta -1 (ruido en DBSCAN)
    valid_data = hado_data[hado_data[label_column] != -1]
    labels = valid_data[label_column]
    true_labels = valid_data['diagnosis_category']

    # Calcular métricas
    ari = adjusted_rand_score(true_labels, labels)
    nmi = normalized_mutual_info_score(true_labels, labels)
    homogeneity = homogeneity_score(true_labels, labels)
    completeness = completeness_score(true_labels, labels)

    # Almacenar resultados
    external_validation_metrics.append({'Method': method,
                                       'Adjusted Rand Index': ari,
                                       'Normalized Mutual Information': nmi,
                                       'Homogeneity': homogeneity,
                                       'Completeness': completeness})

# Convertir a DataFrame para facilitar la visualización
external_validation_metrics_df = pd.DataFrame(external_validation_metrics)
external_validation_metrics_df

,Method,Adjusted Rand Index,Normalized Mutual Information,Homogeneity,Completeness
0,K-means,0.120388,0.319291,0.242589,0.466923
1,LDA,0.138949,0.208401,0.182552,0.242778
2,Hierarchical,0.163972,0.337813,0.258229,0.488304
3,DBSCAN,0.268495,0.663769,0.993976,0.498247


### Resumen de los Resultados

#### Validación Interna
- **DBSCAN** tuvo el mejor rendimiento en términos de la puntuación de Silhouette y el índice de Calinski-Harabasz, lo que sugiere que los clusters son densos y bien separados.
- **K-means y Clustering Jerárquico** tuvieron un rendimiento similar pero inferior al de DBSCAN.
- **LDA** tuvo el peor rendimiento en estas métricas.

#### Validación Externa
- **Clustering Jerárquico** tuvo el mejor rendimiento en términos de índice Rand ajustado y homogeneidad, lo que sugiere que los clusters coinciden bien con las categorías de diagnóstico predefinidas.
- **K-means** también tuvo un buen rendimiento en estas métricas.
- **DBSCAN** tuvo un buen rendimiento en términos de información mutua normalizada y completitud.
- **LDA** tuvo el peor rendimiento en estas métricas.

### Conclusión
- Si la densidad y la separación de los clusters son más importantes, **DBSCAN** parece ser la mejor opción.
- Si la coincidencia con las categorías de diagnóstico predefinidas es más importante, **Clustering Jerárquico** o **K-means** serían más apropiados.
- **LDA** parece ser el menos efectivo para este conjunto de datos específico, tanto en términos de métricas internas como externas.

### Siguiente Paso
Podríamos considerar aplicar técnicas de reducción de dimensionalidad como PCA para visualizar estos clusters en un espacio de menor dimensión o probar otros métodos de clustering para validar aún más la robustez de nuestras interpretaciones.